### Compare new tariffs (Partner, P4, MB, Smart)
Данная методика используется
 - для анализа изменений тарифов
 - для генерации файла сравнения тарифов всех Спонсоров
 - для генерации файлов для загрузки изменений тарифов в OCS
<br><br>
Данная методика позволяет найти:<br>
 - Новые тарифы
 - Пропавшие тарифы
 - Изменившиеся тарифы
 <br><br>
Последнее обновление методики было сделано 22.11.2019<br>
<br>
Для анализа добавленных операторов 
Дополнительно можно использовать методику<br>
!!! 
Step_04_Prepare_and_Load_PP_into_OCS

In [5]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'
eur_usd_rate = 1.15
tariff_tolerance = 0.0001
smart_price_to_cost_rate = 1.15

# Input files with new teriffs

file_p4_new_tariffs = 'P4_20200101.csv'
file_partner_new_tariffs = 'Partner_20200101.csv'
file_mb_new_tariffs = 'MB_20200116.csv'
file_smart_new_tariffs = 'SMART_20200102_COST.csv'
file_smart_new_tariffs_data = 'SMART_DATA_191113.csv' # for files with data only tariffs
file_jt_new_tariffs = 'JT_20200101.csv' # for files with data only tariffs
file_allowed_list = 'DMI.dmi_allowed_list_export_Sun_Jan_12_2020.csv'

# Output files fith results

file_s1_s2_tariffs_compare = 'df_s1_s2_tariffs_compare.csv'

file_s1_to_db = 'df_s1_update_267.csv'
file_s1_to_db_100 = 'df_s1_update_267_100.csv'
file_new_s1_to_db = 'df_s1_insert_new_267.csv'
file_new_s1_to_db_100 = 'df_s1_insert_new_267_100.csv'
file_close_s1_to_db = 'df_s1_close_267.csv'
file_close_s1_to_db_100 = 'df_s1_close_267_100.csv'

file_s2_to_db = 'df_s2_update_268.csv'
file_s2_to_db_100 = 'df_s2_update_268_100.csv'
file_new_s2_to_db = 'df_s2_insert_new_268.csv'
file_new_s2_to_db_100 = 'df_s2_insert_new_268_100.csv'
file_close_s2_to_db = 'df_s2_close_268.csv'
file_close_s2_to_db_100 = 'df_s2_close_268_100.csv'

file_s4_to_db = 'df_s4_update_329.csv'
file_s4_to_db_100 = 'df_s4_update_329_100.csv'
file_new_s4_to_db = 'df_s4_insert_new_329.csv'
file_new_s4_to_db_100 = 'df_s4_insert_new_329_100.csv'
file_close_s4_to_db = 'df_s4_close_329.csv'
file_close_s4_to_db_100 = 'df_s4_close_329_100.csv'

file_s5_to_db = 'df_s5_update.csv'
file_s5_to_db_100 = 'df_s5_update_100.csv'
file_new_s5_to_db_clone = 'df_s5_insert_new_clone.csv'
file_new_s5_to_db_clone_100 = 'df_s5_insert_new_clone_100.csv'
file_new_s5_to_db = 'df_s5_insert_new.csv'
file_new_s5_to_db_100 = 'df_s5_insert_new_100.csv'
file_close_s5_to_db = 'df_s5_close.csv'
file_close_s5_to_db_100 = 'df_s5_close_100.csv'

# Output files for RoamDb
file_s5_roamdb = 'df_s5_roamdb.csv'

### Подготовка данных

In [2]:
# Из базы данных OCS выгружаем TADIG и PP

import pandas as pd
import pyodbc

#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

# Получение данных TADIG

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''
df_tadig = pd.read_sql_query(sql_srt, cnxn)

# Получение тарифов Partner из OCS

sql_srt=\
'''
SELECT
MO_CALL_RATE MOC,
MT_CALL_RATE MTC,
MO_SMS_RATE SMS,
DATA_RATE DATA,
NETWORK_ID,SPONSOR_ID,is_discounted
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=267
AND end_date IS NULL
'''

df_ocs_partner = pd.read_sql_query(sql_srt, cnxn)
print('Partner OCS cost:')
display(df_ocs_partner.head(3))

# Получение тарифов P4 из OCS

# Convert to USD
sql_srt=\
'''
SELECT
IIF(CURRENCY_ID=1,MO_CALL_RATE,MO_CALL_RATE * 1.2) MOC,
IIF(CURRENCY_ID=1,MT_CALL_RATE,MT_CALL_RATE * 1.2) MTC,
IIF(CURRENCY_ID=1,MO_SMS_RATE,MO_SMS_RATE * 1.2) SMS,
IIF(CURRENCY_ID=1,DATA_RATE,DATA_RATE * 1.2) DATA,
NETWORK_ID,SPONSOR_ID,is_discounted
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=268
AND end_date IS NULL
'''

df_ocs_p4 = pd.read_sql_query(sql_srt, cnxn)
print('\nP4 OCS cost:')
display(df_ocs_p4.head(3))

# Получение тарифов MB из OCS

sql_srt=\
'''
SELECT
MO_CALL_RATE MOC,
MT_CALL_RATE MTC,
MO_SMS_RATE SMS,
DATA_RATE DATA,
NETWORK_ID,SPONSOR_ID,is_discounted
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=329
AND end_date IS NULL
'''

df_ocs_mb = pd.read_sql_query(sql_srt, cnxn)
print('MB OCS cost:')
display(df_ocs_mb.head(3))

# Получение тарифов Smart из OCS

sql_srt=\
'''
SELECT
MO_CALL_RATE MOC,
MT_CALL_RATE MTC,
MO_SMS_RATE SMS,
DATA_RATE DATA,
NETWORK_ID,SPONSOR_ID,is_discounted
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=343
AND end_date IS NULL
'''

df_ocs_smart = pd.read_sql_query(sql_srt, cnxn)
print('Smart OCS cost:')
df_ocs_smart[['MOC','MTC','SMS','DATA']] = df_ocs_smart[['MOC','MTC','SMS','DATA']] / 1.15
display(df_ocs_smart.head(3))

# Получение тарифов JT из OCS

sql_srt=\
'''
SELECT
MO_CALL_RATE MOC,
MT_CALL_RATE MTC,
MO_SMS_RATE SMS,
DATA_RATE DATA,
NETWORK_ID,SPONSOR_ID,is_discounted
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=372
AND end_date IS NULL
'''

df_ocs_jt = pd.read_sql_query(sql_srt, cnxn)
print('JT OCS cost:')
display(df_ocs_jt.head(3))

# Получение тарифов Partner, P4, MB, Smart из OCS по прайспланам 267, 268, 343, 322, 323, 324

# 267 Partner Cost Plan 2018
# 321 Partner Charge 2019
# 268 P4 Cost Plan 2018
# 322 P4 Charge 2019
# 329 MB Cost Plan 2019
# 330 MB Charge 2019
# 343 SMART Cost Plan 2019
# 344 SMART Charge 2019
# 372 S8 Cost 2019
# 373 S8 Charge 2019
# 323 Full Combined Charge 2019
# 324 Combined Charge 2019

sql_srt=\
'''
SELECT
*
FROM ROAMING_PLAN_RULES
WHERE roaming_plan_id in (267, 321, 268, 322, 329, 330, 343, 344, 323, 324, 372, 373)
AND end_date IS NULL
'''

df_ocs_all = pd.read_sql_query(sql_srt, cnxn)
print('\nOCS all:')
display(df_ocs_all.head(3))

cnxn.close()

Partner OCS cost:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted
0,0.990086,0.990086,0.249622,10.238187,59.0,1.0,False
1,2.189060,2.189060,0.450132,14.333252,95.0,1.0,False
2,2.367968,0.000000,0.494058,15.786522,113.0,1.0,False



P4 OCS cost:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted
0,0.999,0.999,9.000,6.188959,411.0,2.0,False
1,3.180,0.000,0.768,14.280000,465.0,2.0,False
2,2.160,0.000,0.600,18.000000,1181.0,2.0,False


MB OCS cost:


...

,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted
0,0.14,0.00,0.0075,0.0075,12.0,5.0,True
1,0.35,0.03,0.0100,0.0080,25.0,5.0,True
2,0.25,0.00,0.0100,0.0100,29.0,5.0,True


JT OCS cost:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted
0,0.2052,0.0,0.1128,0.0168,232.0,8.0,True
1,1.0164,0.0,0.1932,0.1044,630.0,8.0,True
2,0.2460,0.0,0.1020,0.0204,845.0,8.0,True



OCS all:


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted,HIDDEN,DailyCap,DailyCapLimit,DailyCapTimeZone
0,5567.0,267.0,59.0,1.0,0.990086,0.990086,0.249622,0.0,10.238187,1.0,2017-11-01,None,False,None,False,NaN,NaN
1,5568.0,267.0,95.0,1.0,2.189060,2.189060,0.450132,0.0,14.333252,1.0,2017-11-01,None,False,None,False,NaN,NaN
2,5571.0,267.0,113.0,1.0,2.367968,0.000000,0.494058,0.0,15.786522,1.0,2017-11-01,None,False,None,False,NaN,NaN


In [6]:
# Загружаем из файлов новые тарифы Partner, P4, MB и Allowed list

print('Новые тарифы Partner:')
df_partner = pd.read_csv(join(load_folder, file_partner_new_tariffs),sep=',')
display(df_partner.head(3))

print('Новые тарифы P4:')
df_p4 = pd.read_csv(join(load_folder, file_p4_new_tariffs),sep=',')
df_p4.Data_Increment_KB.replace({'1 kB':1, '10 kB':10, '1kB':1}, inplace=True)
df_p4.Voice_Increment_SEC.replace({'30/1':30}, inplace=True)
df_p4.MTC.fillna(0, inplace=True)
df_p4[['MOC','MTC','SMS','MB']] = df_p4[['MOC','MTC','SMS','MB']] * eur_usd_rate
display(df_p4.head(3))

print('Новые тарифы MB:')
df_mb = pd.read_csv(join(load_folder, file_mb_new_tariffs),sep=',')
display(df_mb.head(3))

print('Новые тарифы JT:')
df_jt = pd.read_csv(join(load_folder, file_jt_new_tariffs),sep=',')
display(df_jt.head(3))

# Allowed lists
df_al=pd.read_csv(join(load_folder, file_allowed_list),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al = pd.merge(df_al, df_tadig[['TADIG_CODE','NETWORK_ID']], how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al.drop('TADIG_CODE',inplace=True,axis=1)

Новые тарифы Partner:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,ALBVF,27602.0,Vodafone AL,Albania,NaN,0.0300,0.0,0.0060,0.0024,1,1.0
1,ANDMA,21303.0,Andorra Telecom SAU,Andorra,NaN,0.0900,0.0,0.0300,0.0300,1,1.0
2,AIACW,36584.0,AIACW AI,Anguilla,NaN,0.1606,0.0,0.0128,0.0257,60,1.0


Новые тарифы P4:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,P4_CAMEL,P4_LTE
0,Albania,TELEKOM ALBANIA SH.A,ALBAM,0.51635,0.0,0.0598,0.04600,1,1,YES,YES
1,Albania,ALBtelecom sh.a.,ALBEM,0.44850,0.0,0.0460,0.04485,60,1,YES,YES
2,Australia,SingTel Optus Pty Limited,AUSOP,0.06325,0.0,0.0092,0.00920,60,1,YES,YES


Новые тарифы MB:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,LTE
0,HK,3HK,HKGH3,0.013,0.013,0.13,0.00315,60,1,Yes
1,HK,3HK,HKGHT,0.013,0.013,0.13,0.00315,60,1,Yes
2,Macau,3 Macau,MACHT,0.130,0.130,0.13,0.00504,60,1,Yes


Новые тарифы JT:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,LTE
0,NaN,NaN,ABWDC,0.3576,0.0,0.1800,0.0720,NaN,1,No
1,NaN,NaN,AFGAR,1.0164,0.0,0.1236,0.1044,NaN,1,No
2,NaN,NaN,AFGEA,0.7956,0.0,0.2880,0.2976,NaN,1,No


In [8]:
# Загружаем из файлов новые тарифы Smart с очисткой данных

def clean_smart_file(folder, file, tariff_type):
    # tariff_type "data" / "all"
    df_smart = pd.read_csv(join(folder, file),sep=',')
#     df_smart.Data_Increment_KB.replace({'1 kB':1, '10 kB':10, '1kB':1, '1kb':1, '10kb':10, ' 10kb ':10}, inplace=True)
    df_smart.LTE_S5.replace({'3G':'No', '4G':'Yes'}, inplace=True)
    df_smart.Data_Increment_KB.fillna(10, inplace=True)
    df_smart.Voice_Increment_SEC.fillna(60, inplace=True)
    
    # #####################################################
    #Выбрать только первый TADIG из ячейки, если их там несколько через запятую
    # #####################################################
    if df_smart.TADIG.values.size > 0:
        df_smart.TADIG = df_smart.TADIG.str.extract('([A-Z0-9]+)', expand=True)
    
    # #####################################################
    
    if tariff_type == 'data':
        df_smart.MOC.fillna(50, inplace=True)
        df_smart.MTC.fillna(50, inplace=True)
        df_smart.SMS.fillna(50, inplace=True)
    elif tariff_type == 'all':
        
        if df_smart.MOC.dtypes != 'float64':
            df_smart.loc[df_smart.MOC.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'MOC'] =\
            df_smart.MOC.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64') * eur_usd_rate

        if df_smart.MTC.dtypes != 'float64':
            df_smart.loc[df_smart.MTC.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'MTC'] =\
            df_smart.MTC.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64') * eur_usd_rate
            
        df_smart.MTC.fillna(0, inplace=True)

        if df_smart.SMS.dtypes != 'float64':
            df_smart.loc[df_smart.SMS.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'SMS'] =\
            df_smart.SMS.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64') * eur_usd_rate
            
        df_smart.MOC.fillna(50, inplace=True)

    # #####################################################
    # Очистить данные от EUR с конвертацией в USD
    # #####################################################

    if df_smart.MB.dtypes != 'float64':
        df_smart.loc[df_smart.MB.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'MB'] =\
        df_smart.MB.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64') * eur_usd_rate

    df_smart[['MOC','MTC','SMS','MB']] = df_smart[['MOC','MTC','SMS','MB']].replace(to_replace={r'-':0}, regex=True)

    df_smart[['MOC','MTC','SMS','MB']] = df_smart[['MOC','MTC','SMS','MB']].astype('float64')
    
    return df_smart

df_smart = clean_smart_file(load_folder, file_smart_new_tariffs, 'all')

# df_smart_data = clean_smart_file(load_folder, file_smart_new_tariffs_data, "data")

df_smart_data = DataFrame({'TADIG':[]})

df_smart_all = df_smart.append(df_smart_data[~df_smart_data.TADIG.isin(df_smart.TADIG.tolist())], ignore_index=True, sort=False)

# df_smart_all[['MOC','MTC','SMS','MB']] = df_smart_all[['MOC','MTC','SMS','MB']] * smart_price_to_cost_rate

df_smart_all.to_csv(join(load_folder, 'SMART_COST.csv'), index=False)
df_smart_all.head(3)

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,LTE_S5
0,ALBANIA,VODAFONE ALBANIA,ALBVF,0.14,0.00,0.0075,0.0075,60.0,1.0,No
1,ANGUILLA,C&W Group,AIAK9,0.50,0.05,0.0500,0.0500,60.0,1.0,No
2,ANTIGUA,C&W Group,ATGK9,0.50,0.05,0.0500,0.0500,60.0,1.0,No


### Проверка данных
Следующим шагом выполняется проверка данных и генерация сводного файла сравнения всех Спонсоров

**!!! Если нужно сделать сравнение текущих тарифов друг с другом (тех, которые сейчас в OCS), нужно выгрузить их в файлы !!!**
<br><br>
Для генерации сводного файла используются тарифы из файлов, а не загружаемые из OCS.
<br><br>
Запросы из OCS, для получения файлов тарифов:<br>
Partner:<br>
SELECT t.tadig_code AS TADIG
,'' AS MCCMNC, '' AS PARTNER, '' AS COUNTRY, '' AS Operator
,MO_CALL_RATE MOC,MT_CALL_RATE MTC,MO_SMS_RATE SMS,DATA_RATE MB,
'' AS Voice_Increment_SEC, '' AS Data_Increment_KB FROM ROAMING_PLAN_RULES r
LEFT JOIN RDB_TADIG_CODES t
ON r.network_id=t.network_id
LEFT JOIN ROAMING_PLANS p
ON r.roaming_plan_id=p.roaming_plan_id
WHERE r.roaming_plan_id IN (267)
AND r.SPONSOR_ID = 1
AND p.deleted IS NULL
AND r.end_date IS NULL

In [9]:
# Объединяем !!! новые !!! тарифы с TADIG и Allowed list

# Добавляем к новым тарифам TADIG из OCS и находим отсутствующие в BD OCS TADIG в новых тарифах Partner

df_partner_net_id = pd.merge(df_partner, df_tadig, left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Отсутствующие в BD OCS TADIG в новых тарифах Partner:')
display(df_partner_net_id[df_partner_net_id.TADIG_CODE.isnull()])

# Добавляем к новым тарифам TADIG из OCS и находим отсутствующие в BD OCS TADIG в новых тарифах P4

df_p4_net_id = pd.merge(df_p4,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Отсутствующие в BD OCS TADIG в новых тарифах P4:')
display(df_p4_net_id[df_p4_net_id.TADIG_CODE.isnull()])

# Добавляем к новым тарифам TADIG из OCS и находим отсутствующие в BD OCS TADIG в новых тарифах MB

df_mb_net_id = pd.merge(df_mb,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Отсутствующие в BD OCS TADIG в новых тарифах MB:')
display(df_mb_net_id[df_mb_net_id.TADIG_CODE.isnull()])

# Добавляем к новым тарифам TADIG из OCS и находим отсутствующие в BD OCS TADIG в новых тарифах Smart

df_smart_net_id = pd.merge(df_smart_all, df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Отсутствующие в BD OCS TADIG в новых тарифах Smart:')
display(df_smart_net_id[df_smart_net_id.TADIG_CODE.isnull()])

# Добавляем к новым тарифам TADIG из OCS и находим отсутствующие в BD OCS TADIG в новых тарифах JT

df_jt_net_id = pd.merge(df_jt,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Отсутствующие в BD OCS TADIG в новых тарифах JT:')
display(df_jt_net_id[df_jt_net_id.TADIG_CODE.isnull()])

# Объединяем тарифы S1, S2, S4 полученные на предыдущех шагах, в один DF по NETWORK_ID

df_s1_s2_temp = pd.merge(df_partner_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                              df_p4_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','P4_CAMEL','P4_LTE',
                                  'Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID'],suffixes=('_S1','_S2')).drop_duplicates()

df_s1_s2_s4_temp = pd.merge(df_s1_s2_temp,
                              df_mb_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','LTE',
                                  'Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID']).drop_duplicates()

df_s1_s2_s4_temp.rename({'MOC':'MOC_S4','MTC':'MTC_S4','SMS':'SMS_S4','MB':'MB_S4','LTE':'LTE_S4',
                        'Voice_Increment_SEC':'Voice_Increment_SEC_S4','Data_Increment_KB':'Data_Increment_KB_S4'},
                         axis=1,inplace=True)

df_s1_s2_s4_s5_temp = pd.merge(df_s1_s2_s4_temp,
                              df_smart_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','LTE_S5',
                                  'Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID']).drop_duplicates()

df_s1_s2_s4_s5_temp.rename({'MOC':'MOC_S5','MTC':'MTC_S5','SMS':'SMS_S5','MB':'MB_S5',
                        'Voice_Increment_SEC':'Voice_Increment_SEC_S5','Data_Increment_KB':'Data_Increment_KB_S5'},
                         axis=1,inplace=True)

df_s1_s2_s4_s5_temp = pd.merge(df_s1_s2_s4_s5_temp,
                              df_jt_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','LTE',
                                  'Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID']).drop_duplicates()


df_s1_s2_s4_s5_temp.rename({'MOC':'MOC_S8','MTC':'MTC_S8','SMS':'SMS_S8','MB':'MB_S8','LTE':'LTE_S8',
                        'Voice_Increment_SEC':'Voice_Increment_SEC_S8','Data_Increment_KB':'Data_Increment_KB_S8'},
                         axis=1,inplace=True)

# Добавляем к DF с тарифами S1, S2, S4, s5 полученному на предыдущем шаге, TADIG из OCS по NETWORK_ID

df_s1_s2_s4 = pd.merge(df_s1_s2_s4_s5_temp, df_tadig, how='left',on='NETWORK_ID')

# Добавляем Спонсора из Allowed list и отмечаем в какой стране какой Спонсор для Coombined AL

df_s1_s2_s4_sponsor = pd.merge(df_s1_s2_s4, df_al, how='outer', on='NETWORK_ID')
df_s1_s2_s4_sponsor['COUNTRY_SPONSOR'] = df_s1_s2_s4_sponsor.groupby('COUNTRY_NAME')['SPONSOR'].transform('any')

# Результирующий файл c указанием страны принадлежности Спонсора

f_list_for_file = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE'
          ,'MOC_S1','MOC_S2','MOC_S4','MOC_S5','MOC_S8','MTC_S1','MTC_S2','MTC_S4','MTC_S5','MTC_S8'
          ,'SMS_S1','SMS_S2','SMS_S4','SMS_S5','SMS_S8'
          ,'MB_S1','MB_S2','MB_S4','MB_S5','MB_S8','SPONSOR','COUNTRY_SPONSOR'
          ,'P4_CAMEL','P4_LTE','LTE_S4','LTE_S5','LTE_S8']

f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2',
          'MOC_S4','MTC_S4','SMS_S4','MB_S4',
          'MOC_S5','MTC_S5','SMS_S5','MB_S5',
          'MOC_S8','MTC_S8','SMS_S8','MB_S8',
          'SPONSOR','COUNTRY_SPONSOR','TADIG_CODE_AL','NETWORK_ID','P4_CAMEL','P4_LTE','LTE_S4','LTE_S5','LTE_S8'
         ,'Voice_Increment_SEC_S1','Data_Increment_KB_S1','Voice_Increment_SEC_S2','Data_Increment_KB_S2'
         ,'Voice_Increment_SEC_S4','Data_Increment_KB_S4','Voice_Increment_SEC_S5','Data_Increment_KB_S5'
         ,'Voice_Increment_SEC_S8','Data_Increment_KB_S8']

df_s1_s2_s4_sponsor = df_s1_s2_s4_sponsor[f_list].drop_duplicates()
df_s1_s2_s4_sponsor[f_list_for_file].to_csv(join(load_folder, file_s1_s2_tariffs_compare), index=False)
print('\nВсе тарифы с привязкой к Allowed list:')
df_s1_s2_s4_sponsor.head(3)

Отсутствующие в BD OCS TADIG в новых тарифах Partner:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID


Отсутствующие в BD OCS TADIG в новых тарифах P4:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,P4_CAMEL,P4_LTE,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
83,Kosovo,IPKO Telecommunications LLC,K0001,0.529,0.0,0.013225,0.066125,1,1,YES,YES,NaN,NaN,NaN,NaN
102,Mozambique,MOVITEL S.A,MOZVG,0.805,0.0,0.080500,0.048300,60,10,YES,NO,NaN,NaN,NaN,NaN


Отсутствующие в BD OCS TADIG в новых тарифах MB:


...

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,LTE_S5,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
1,ANGUILLA,C&W Group,AIAK9,0.500,0.0500,0.0500,0.05000,60.0,1.0,No,NaN,NaN,NaN,NaN
2,ANTIGUA,C&W Group,ATGK9,0.500,0.0500,0.0500,0.05000,60.0,1.0,No,NaN,NaN,NaN,NaN
21,BERMUDA,Bermuda Digital,BMUK9,0.250,0.0300,0.0200,0.01000,60.0,1.0,No,NaN,NaN,NaN,NaN
38,CANADA,Eastlink,CANK7,0.300,0.0100,0.0100,0.01000,60.0,1.0,No,NaN,NaN,NaN,NaN
40,CAYMAN,C&W Group,CYMK9,0.500,0.0500,0.0500,0.05000,60.0,1.0,No,NaN,NaN,NaN,NaN
57,DOMINICA,C&W Group,DMAK9,0.500,0.0500,0.0500,0.05000,60.0,1.0,No,NaN,NaN,NaN,NaN
65,FINLAND,DNA plc,FINTA,0.575,0.0575,0.1150,0.00575,60.0,1.0,No,NaN,NaN,NaN,NaN
78,GRENADA,C&W Group,GRDK9,0.500,0.0500,0.0500,0.05000,60.0,1.0,No,NaN,NaN,NaN,NaN
79,GUAM,IT&E,USAGU,0.500,0.0000,0.0100,0.00500,60.0,1.0,Yes,NaN,NaN,NaN,NaN
81,GUYANA,Guyana Telephone & Telegraph,GUYK8,0.250,0.0300,0.0200,0.01000,60.0,1.0,No,NaN,NaN,NaN,NaN


Отсутствующие в BD OCS TADIG в новых тарифах JT:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,LTE,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
23,NaN,NaN,BDIL1,0.8928,0.0000,0.2700,0.3552,NaN,1,No,NaN,NaN,NaN,NaN
34,NaN,NaN,BHSNC,1.3368,0.0000,0.1980,0.2136,NaN,1,Yes,NaN,NaN,NaN,NaN
39,NaN,NaN,BLZSC,0.8928,0.0900,0.1800,0.0900,NaN,1,Yes,NaN,NaN,NaN,NaN
62,NaN,NaN,CHNCM,0.4992,0.2496,0.1236,0.0144,NaN,1,Yes,NaN,NaN,NaN,NaN
107,NaN,NaN,FINTA,0.1644,0.0000,0.1332,0.2460,NaN,1,No,NaN,NaN,NaN,NaN
129,NaN,NaN,GINAG,1.0164,0.0000,0.1236,0.1044,NaN,1,No,NaN,NaN,NaN,NaN
131,NaN,NaN,GNQHT,0.8028,0.0000,0.2700,0.2676,NaN,1,No,NaN,NaN,NaN,NaN
198,NaN,NaN,KIRKL,4.9632,0.0000,0.2544,0.1644,NaN,1,No,NaN,NaN,NaN,NaN
222,NaN,NaN,MCOM1,1.0164,0.0000,0.1332,0.2052,NaN,1,No,NaN,NaN,NaN,NaN
228,NaN,NaN,MEXN3,0.3576,0.0900,0.2976,0.3552,NaN,1,No,NaN,NaN,NaN,NaN



Все тарифы с привязкой к Allowed list:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,MOC_S2,MTC_S2,SMS_S2,...,Voice_Increment_SEC_S1,Data_Increment_KB_S1,Voice_Increment_SEC_S2,Data_Increment_KB_S2,Voice_Increment_SEC_S4,Data_Increment_KB_S4,Voice_Increment_SEC_S5,Data_Increment_KB_S5,Voice_Increment_SEC_S8,Data_Increment_KB_S8
0,Albania,Vodafone,ALBVF,0.0300,0.0,0.0060,0.0024,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,60.0,1.0,NaN,1.0
1,Andorra,Andorra Telecom S A U,ANDMA,0.0900,0.0,0.0300,0.0300,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,NaN,NaN,NaN,...,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [10]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

print('Partner:')
df_temp = df_partner_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df_temp[df_temp.values > 1]
display(s1)

print('\nP4:')
df_temp = df_p4_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s2 = df_temp[df_temp.values > 1]
display(s2)

print('\nMB:')
df_temp = df_mb_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s4 = df_temp[df_temp.values > 1]
display(s4)

print('\nSmart:')
df_temp = df_smart_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s5 = df_temp[df_temp.values > 1]
display(s5)

print('\nJT:')
df_temp = df_jt_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s8 = df_temp[df_temp.values > 1]
display(s8)

Partner:


,MOC,MTC,SMS,MB
NETWORK_ID,,,,



P4:


,MOC,MTC,SMS,MB
NETWORK_ID,,,,



MB:


,MOC,MTC,SMS,MB
NETWORK_ID,,,,



Smart:


,MOC,MTC,SMS,MB
NETWORK_ID,,,,



JT:


,MOC,MTC,SMS,MB
NETWORK_ID,,,,


### Сравнение данных
Сравнение старых тарифов, полученных из OCS, с новыми тарифами, загруженными из файлов.

In [11]:
# Проверить изменения тарифов P4

df_temp = pd.merge(df_s1_s2_s4_sponsor, df_ocs_p4, how='outer', on='NETWORK_ID')

# Проверить, есть ли новые PLMN

print("Новые TADIG P4:")
cols_p4 = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S2','MTC_S2','SMS_S2','MB_S2'
           ,'MOC','MTC','SMS','DATA','P4_LTE','P4_CAMEL','is_discounted','NETWORK_ID','SPONSOR']
df_new_s2 = df_temp.loc[((df_temp.MB_S2.notnull()) & (df_temp.SPONSOR_ID.isnull()) & (df_temp.TADIG_CODE.notnull())), cols_p4]
display(df_new_s2)


# Проверить, не закрылись ли какие-нибудь сети

print("\nПропавшие TADIG P4:")
s2 = df_temp[(df_temp.MB_S2.isnull()) & (df_temp.SPONSOR_ID.notnull()) & (df_temp.is_discounted == True)].NETWORK_ID.values
df_closed_s2 = pd.merge(df_tadig[df_tadig.NETWORK_ID.isin(s2)], df_temp[['NETWORK_ID','SPONSOR']], how='left', on='NETWORK_ID')

display(df_closed_s2.groupby(['COUNTRY_NAME','NETWORK_NAME'], as_index=False)['TADIG_CODE','NETWORK_ID','SPONSOR'].first())

# Отличия в тарифах P4

print("\nОтличия в тарифах P4:")

#df2=df.loc[((df.MB_S2.notnull()) | (df.DATA.notnull())) & (df.is_discounted == False), cols_p4].copy()
df2 = df_temp.loc[((df_temp.MB_S2.notnull()) | (df_temp.DATA.notnull())) & (df_temp.is_discounted.notnull()), cols_p4].copy()

df_diff_s2 = df2[(df2.MB_S2.notnull()) & (\
     ~np.isclose(df2['MOC_S2'], df2['MOC'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df2['MTC_S2'], df2['MTC'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df2['SMS_S2'], df2['SMS'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df2['MB_S2'], df2['DATA'], atol=tariff_tolerance, equal_nan=True)
)].copy()

def colorize(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    color_list = []
    for tariff_type in [['MOC','MOC_S2'], ['MTC','MTC_S2'], ['SMS','SMS_S2'], ['DATA','MB_S2']]:
        if np.isclose(s[tariff_type[0]], s[tariff_type[1]], atol=tariff_tolerance, equal_nan=True):
            color_list.extend(['', ''])
        elif s[tariff_type[0]] > s[tariff_type[1]]:
            color_list.extend(['background-color: yellow', 'background-color: #81EF21'])
        else:
            color_list.extend(['background-color: yellow', 'background-color: red'])
    return color_list

cols = ['COUNTRY_NAME','TADIG_CODE','MOC','MOC_S2','MTC','MTC_S2','SMS','SMS_S2','DATA','MB_S2'
           ,'P4_LTE','P4_CAMEL','is_discounted','NETWORK_ID','SPONSOR']

display(df_diff_s2[cols].style.apply(colorize, subset=['MOC','MOC_S2','MTC','MTC_S2','SMS','SMS_S2','DATA','MB_S2'], axis=1))

Новые TADIG P4:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S2,MTC_S2,SMS_S2,MB_S2,MOC,MTC,SMS,DATA,P4_LTE,P4_CAMEL,is_discounted,NETWORK_ID,SPONSOR
259,Thailand,True Move H Universal Communication Co Ltd,THACA,0.69,0.0,0.0575,0.0368,NaN,NaN,NaN,NaN,NO,YES,NaN,1221.0,NaN
260,Thailand,True Move H Universal Communication Co Ltd,THACO,0.69,0.0,0.0575,0.0368,NaN,NaN,NaN,NaN,NO,YES,NaN,1221.0,NaN



Пропавшие TADIG P4:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID,SPONSOR



Отличия в тарифах P4:


,COUNTRY_NAME,TADIG_CODE,MOC,MOC_S2,MTC,MTC_S2,SMS,SMS_S2,DATA,MB_S2,P4_LTE,P4_CAMEL,is_discounted,NETWORK_ID,SPONSOR


In [18]:
df_new_s1.to_excel(join(load_folder, 'Partner_New.xlsx'), 'Sheet1', index=False)

In [12]:
# Проверить изменения тарифов Partner

df_temp=pd.merge(df_s1_s2_s4_sponsor, df_ocs_partner, how='outer', on='NETWORK_ID')

# Проверить, есть ли новые PLMN

print("Новые TADIG Partner:")
cols_partner = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE' ,'MOC','MOC_S1','MTC','MTC_S1','SMS','SMS_S1','DATA','MB_S1'
               ,'is_discounted','NETWORK_ID','SPONSOR']

df_new_s1 = df_temp.loc[((df_temp.MB_S1.notnull()) & (df_temp.SPONSOR_ID.isnull())), cols_partner]
display(df_new_s1)
df_new_s1.to_excel(join(load_folder, 'Partner_New.xlsx'), 'Sheet1', index=False)

# Проверить, не закрылись ли какие-нибудь сети

print("\nПропавшие TADIG Partner:")
s1 = df_temp[(df_temp.MB_S1.isnull()) & (df_temp.SPONSOR_ID.notnull()) & (df_temp.is_discounted == True)].NETWORK_ID.values
df_closed_s1 = pd.merge(df_tadig[df_tadig.NETWORK_ID.isin(s1)], df_temp[['NETWORK_ID','SPONSOR']], how='left', on='NETWORK_ID')
display(df_closed_s1)

# Отличия в тарифах

print("\nОтличия в тарифах Partner:")

df1 = df_temp.loc[((df_temp.MB_S1.notnull()) | (df_temp.DATA.notnull())) & (df_temp.is_discounted.notnull())
                  , cols_partner].copy()

df_diff_s1 = df1[(df1.MB_S1.notnull()) & (\
     ~np.isclose(df1['MOC_S1'], df1['MOC'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df1['MTC_S1'], df1['MTC'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df1['SMS_S1'], df1['SMS'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df1['MB_S1'], df1['DATA'], atol=tariff_tolerance, equal_nan=True)
)].copy()

def colorize(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    color_list = []
    for tariff_type in [['MOC','MOC_S1'], ['MTC','MTC_S1'], ['SMS','SMS_S1'], ['DATA','MB_S1']]:
        if np.isclose(s[tariff_type[0]], s[tariff_type[1]], atol=tariff_tolerance, equal_nan=True):
            color_list.extend(['', ''])
        elif s[tariff_type[0]] > s[tariff_type[1]]:
            color_list.extend(['background-color: yellow', 'background-color: #81EF21'])
        else:
            color_list.extend(['background-color: yellow', 'background-color: red'])
    return color_list

cols = ['COUNTRY_NAME','TADIG_CODE','MOC','MOC_S1','MTC','MTC_S1','SMS','SMS_S1','DATA','MB_S1'
           ,'is_discounted','NETWORK_ID','SPONSOR']

display(df_diff_s1[cols].style.apply(colorize, subset=['MOC','MOC_S1','MTC','MTC_S1','SMS','SMS_S1','DATA','MB_S1'], axis=1))

df_diff_s1[cols].style.apply(colorize, subset=['MOC','MOC_S1','MTC','MTC_S1','SMS','SMS_S1','DATA','MB_S1'], axis=1)\
.to_excel(join(load_folder, 'Partner_Difference.xlsx'), 'Sheet1', index=False)
# display(df_diff_s1)

Новые TADIG Partner:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC,MOC_S1,MTC,MTC_S1,SMS,SMS_S1,DATA,MB_S1,is_discounted,NETWORK_ID,SPONSOR



Пропавшие TADIG Partner:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID,SPONSOR



Отличия в тарифах Partner:


,COUNTRY_NAME,TADIG_CODE,MOC,MOC_S1,MTC,MTC_S1,SMS,SMS_S1,DATA,MB_S1,is_discounted,NETWORK_ID,SPONSOR


In [32]:
country_list = ['Mexico',
'Brazil',
'Argentina',
'Peru',
'Ecuador',
'Chile',
'Albania',
'Indonesia',
'Uruguay',
'Dominican Republic',
'Nicaragua',
'Guatemala',
'Lesotho',]
l1 = df_diff_s1.loc[df_diff_s1.COUNTRY_NAME.isin(country_list)].TADIG_CODE.unique().tolist()
l2 = df_new_s1.loc[df_new_s1.COUNTRY_NAME.isin(country_list)].TADIG_CODE.unique().tolist()
list(set(l1+l2))

['LSOVL',
 'GTMTG',
 'GTMSC',
 'PERTM',
 'URYAM',
 'ARGTP',
 'ALBAM',
 'BRACL',
 'ECUPG',
 'ARGCM',
 'CHLSM',
 'IDN89',
 'NICEN',
 'DOMCL',
 'MEXTL',
 'ARGVO']

In [13]:
# Проверить изменения тарифов MB

df_temp = pd.merge(df_s1_s2_s4_sponsor, df_ocs_mb, how='outer', on='NETWORK_ID')

# Проверить, есть ли новые PLMN

print("Новые TADIG MB:")
cols_mb = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S4','MTC_S4','SMS_S4','MB_S4'
           ,'MOC','MTC','SMS','DATA','LTE_S4','is_discounted','NETWORK_ID','SPONSOR']
df_new_s4 = df_temp.loc[((df_temp.MB_S4.notnull()) & (df_temp.SPONSOR_ID.isnull()) & (df_temp.TADIG_CODE.notnull())), cols_mb]
display(df_new_s4)


# Проверить, не закрылись ли какие-нибудь сети

print("\nПропавшие TADIG MB:")
s4 = df_temp[(df_temp.MB_S4.isnull()) & (df_temp.SPONSOR_ID.notnull()) & (df_temp.is_discounted == True)].NETWORK_ID.values
df_closed_s4 = pd.merge(df_tadig[df_tadig.NETWORK_ID.isin(s4)], df_temp[['NETWORK_ID','SPONSOR']], how='left', on='NETWORK_ID')

display(df_closed_s4)

# Отличия в тарифах MB

print("\nОтличия в тарифах MB:")

df4 = df_temp.loc[((df_temp.MB_S4.notnull()) | (df_temp.DATA.notnull())) & (df_temp.is_discounted.notnull()), cols_mb].copy()

df_diff_s4 = df4[(df4.MB_S4.notnull()) & (\
     ~np.isclose(df4['MOC_S4'], df4['MOC'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df4['MTC_S4'], df4['MTC'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df4['SMS_S4'], df4['SMS'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df4['MB_S4'], df4['DATA'], atol=tariff_tolerance, equal_nan=True)
)].copy()

def colorize(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    color_list = []
    for tariff_type in [['MOC','MOC_S4'], ['MTC','MTC_S4'], ['SMS','SMS_S4'], ['DATA','MB_S4']]:
        if np.isclose(s[tariff_type[0]], s[tariff_type[1]], atol=tariff_tolerance, equal_nan=True):
            color_list.extend(['', ''])
        elif s[tariff_type[0]] > s[tariff_type[1]]:
            color_list.extend(['background-color: yellow', 'background-color: #81EF21'])
        else:
            color_list.extend(['background-color: yellow', 'background-color: red'])
    return color_list

cols = ['COUNTRY_NAME','TADIG_CODE','MOC','MOC_S4','MTC','MTC_S4','SMS','SMS_S4','DATA','MB_S4'
           ,'LTE_S4','is_discounted','NETWORK_ID','SPONSOR']

display(df_diff_s4[cols].style.apply(colorize, subset=['MOC','MOC_S4','MTC','MTC_S4','SMS','SMS_S4','DATA','MB_S4'], axis=1))

Новые TADIG MB:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S4,MTC_S4,SMS_S4,MB_S4,MOC,MTC,SMS,DATA,LTE_S4,is_discounted,NETWORK_ID,SPONSOR
982,Malaysia,CelCom,MYSCC,0.26,0.26,0.26,0.002504,NaN,NaN,NaN,NaN,Yes,NaN,580.0,NaN
983,Malaysia,CelCom,MYSMR,0.26,0.26,0.26,0.002504,NaN,NaN,NaN,NaN,Yes,NaN,580.0,NaN
984,Malaysia,CelCom,MYSCC,0.26,0.26,0.26,0.002504,NaN,NaN,NaN,NaN,Yes,NaN,580.0,NaN
985,Malaysia,CelCom,MYSMR,0.26,0.26,0.26,0.002504,NaN,NaN,NaN,NaN,Yes,NaN,580.0,NaN



Пропавшие TADIG MB:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID,SPONSOR
0,Guam,Docomo,GUMDP,360.0,NaN
1,Guam,Docomo,GUMHT,360.0,NaN



Отличия в тарифах MB:


,COUNTRY_NAME,TADIG_CODE,MOC,MOC_S4,MTC,MTC_S4,SMS,SMS_S4,DATA,MB_S4,LTE_S4,is_discounted,NETWORK_ID,SPONSOR
259,Thailand,THACA,0.26,0.26,0.26,0.26,0.26,0.26,0.00756048,0.008,Yes,True,1221,nan
260,Thailand,THACO,0.26,0.26,0.26,0.26,0.26,0.26,0.00756048,0.008,Yes,True,1221,nan
971,Thailand,THACT,0.26,0.26,0.26,0.26,0.26,0.26,0.00756048,0.008,Yes,True,955,nan
974,Malaysia,MYSMI,0.26,0.26,0.26,0.26,0.26,0.26,0.01,0.003,Yes,True,588,nan


In [11]:
# Проверить изменения тарифов Smart

df_temp = pd.merge(df_s1_s2_s4_sponsor, df_ocs_smart, how='outer', on='NETWORK_ID')

# Проверить, есть ли новые PLMN

print("Новые TADIG SMART:")
cols_smart = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S5','MTC_S5','SMS_S5','MB_S5'
           ,'MOC','MTC','SMS','DATA','LTE_S5','is_discounted','NETWORK_ID','SPONSOR']
df_new_s5 = df_temp.loc[((df_temp.MB_S5.notnull()) & (df_temp.SPONSOR_ID.isnull()) & (df_temp.TADIG_CODE.notnull())), cols_smart]
display(df_new_s5)


# Проверить, не закрылись ли какие-нибудь сети

print("\nПропавшие TADIG SMART:")
s5 = df_temp[(df_temp.MB_S5.isnull()) & (df_temp.SPONSOR_ID.notnull()) & (df_temp.is_discounted == True)].NETWORK_ID.values
df_closed_s5 = pd.merge(df_tadig[df_tadig.NETWORK_ID.isin(s5)], df_temp[['NETWORK_ID','SPONSOR']], how='left', on='NETWORK_ID')

display(df_closed_s5)

# Отличия в тарифах SMART

print("\nОтличия в тарифах SMART:")

df5 = df_temp.loc[((df_temp.MB_S5.notnull()) | (df_temp.DATA.notnull())) & (df_temp.is_discounted.notnull()), cols_smart].copy()

df_diff_s5 = df5[(df5.MB_S5.notnull()) & (\
     ~np.isclose(df5['MOC_S5'], df5['MOC'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df5['MTC_S5'], df5['MTC'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df5['SMS_S5'], df5['SMS'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df5['MB_S5'], df5['DATA'], atol=tariff_tolerance, equal_nan=True)
)].copy()

def colorize(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    color_list = []
    for tariff_type in [['MOC','MOC_S5'], ['MTC','MTC_S5'], ['SMS','SMS_S5'], ['DATA','MB_S5']]:
        if np.isclose(s[tariff_type[0]], s[tariff_type[1]], atol=tariff_tolerance, equal_nan=True):
            color_list.extend(['', ''])
        elif s[tariff_type[0]] > s[tariff_type[1]]:
            color_list.extend(['background-color: yellow', 'background-color: #81EF21'])
        else:
            color_list.extend(['background-color: yellow', 'background-color: red'])
    return color_list

cols = ['COUNTRY_NAME','TADIG_CODE','MOC','MOC_S5','MTC','MTC_S5','SMS','SMS_S5','DATA','MB_S5'
           ,'LTE_S5','is_discounted','NETWORK_ID','SPONSOR']

df_diff_s5[cols].style.apply(colorize, subset=['MOC','MOC_S5','MTC','MTC_S5','SMS','SMS_S5','DATA','MB_S5'], axis=1)\
.to_excel(join(load_folder, 'smart_changes.xlsx'), 'Sheet1', index=False)
display(df_diff_s5[cols].style.apply(colorize, subset=['MOC','MOC_S5','MTC','MTC_S5','SMS','SMS_S5','DATA','MB_S5'], axis=1))

Новые TADIG SMART:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S5,MTC_S5,SMS_S5,MB_S5,MOC,MTC,SMS,DATA,LTE_S5,is_discounted,NETWORK_ID,SPONSOR
958,Romania,Orange,ROMMR,0.166,0.0,0.011,0.0055,NaN,NaN,NaN,NaN,No,NaN,759.0,NaN



Пропавшие TADIG SMART:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID,SPONSOR
0,Japan,NTT Docomo Inc,JPNDO,499.0,S1



Отличия в тарифах SMART:


,COUNTRY_NAME,TADIG_CODE,MOC,MOC_S5,MTC,MTC_S5,SMS,SMS_S5,DATA,MB_S5,LTE_S5,is_discounted,NETWORK_ID,SPONSOR
9,Australia,AUSOP,0.5,0.2,0,0,0.15,0.005,0.45,0.002,No,True,45,S1
37,Canada,CANBM,0.25,0.2,0,0,0.01,0.005,0.02,0.003,Yes,True,150,S1
38,Canada,CANTS,0.25,0.2,0,0,0.01,0.005,0.02,0.003,Yes,True,167,S1
44,China,CHNCT,0.45,0.45,0.02,0.02,0.1,0.05,0.007,0.0025,Yes,True,196,S1
60,Denmark,DNKDM,0.5927,0.575,0.0593,0.0575,0.1185,0.115,0.0083,0.00575,Yes,True,248,S2
61,Denmark,DNKT2,0.5927,0.575,0.0593,0.0575,0.1185,0.115,0.0083,0.00575,Yes,True,248,S2
65,Estonia,ESTEM,0.27,0.26,0.023,0.011,0.012,0.0088,0.007,0.002048,No,True,272,S2
69,Finland,FINTF,0.27,0.26,0.023,0.011,0.012,0.0088,0.007,0.002048,No,True,291,S2
73,France,FRA09,0.2,0.2,0,0,0.02,0.01,0.01,0.007,Yes,True,305,S2
74,France,FRAF2,0.2,0.2,0,0,0.02,0.01,0.01,0.007,Yes,True,305,S2


In [18]:
df_diff_s5.TADIG_CODE.values.tolist()

['AUSOP',
 'CANBM',
 'CANTS',
 'CHNCT',
 'DNKDM',
 'DNKT2',
 'ESTEM',
 'FINTF',
 'FRA09',
 'FRAF2',
 'NORNC',
 'NORTM',
 'SWECN',
 'SWEEP',
 'THADT',
 'BELMO',
 'ISLTL',
 'ISRPL',
 'LVABT',
 'LVAK9',
 'LTUMT',
 'LUXVM',
 'SVKGT',
 'AUSTA',
 'CANST',
 'FIN2G',
 'HKGPP',
 'ISRCL',
 'MYSMT',
 'MMRPT',
 'NGAMN',
 'NORNN',
 'THAWP',
 'TONDP']

In [15]:
# Проверить изменения тарифов JT

df_temp = pd.merge(df_s1_s2_s4_sponsor, df_ocs_jt, how='outer', on='NETWORK_ID')

# Проверить, есть ли новые PLMN

print("Новые TADIG JT:")
cols_jt = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S8','MTC_S8','SMS_S8','MB_S8'
           ,'MOC','MTC','SMS','DATA','LTE_S8','is_discounted','NETWORK_ID','SPONSOR']
df_new_s8 = df_temp.loc[((df_temp.MB_S8.notnull()) & (df_temp.SPONSOR_ID.isnull()) & (df_temp.TADIG_CODE.notnull())), cols_jt]
display(df_new_s8)


# Проверить, не закрылись ли какие-нибудь сети

print("\nПропавшие TADIG JT:")
s8 = df_temp[(df_temp.MB_S8.isnull()) & (df_temp.SPONSOR_ID.notnull()) & (df_temp.is_discounted == True)].NETWORK_ID.values
df_closed_s8 = pd.merge(df_tadig[df_tadig.NETWORK_ID.isin(s8)], df_temp[['NETWORK_ID','SPONSOR']], how='left', on='NETWORK_ID')

display(df_closed_s8)

# Отличия в тарифах JT

print("\nОтличия в тарифах JT:")

df8 = df_temp.loc[((df_temp.MB_S8.notnull()) | (df_temp.DATA.notnull())) & (df_temp.is_discounted.notnull()), cols_jt].copy()

df_diff_s8 = df8[(df8.MB_S8.notnull()) & (\
     ~np.isclose(df8['MOC_S8'], df8['MOC'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df8['MTC_S8'], df8['MTC'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df8['SMS_S8'], df8['SMS'], atol=tariff_tolerance, equal_nan=True) |\
     ~np.isclose(df8['MB_S8'], df8['DATA'], atol=tariff_tolerance, equal_nan=True)
)].copy()

def colorize(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    color_list = []
    for tariff_type in [['MOC','MOC_S8'], ['MTC','MTC_S8'], ['SMS','SMS_S8'], ['DATA','MB_S8']]:
        if np.isclose(s[tariff_type[0]], s[tariff_type[1]], atol=tariff_tolerance, equal_nan=True):
            color_list.extend(['', ''])
        elif s[tariff_type[0]] > s[tariff_type[1]]:
            color_list.extend(['background-color: yellow', 'background-color: #81EF21'])
        else:
            color_list.extend(['background-color: yellow', 'background-color: red'])
    return color_list

cols = ['COUNTRY_NAME','TADIG_CODE','MOC','MOC_S8','MTC','MTC_S8','SMS','SMS_S8','DATA','MB_S8'
           ,'LTE_S8','is_discounted','NETWORK_ID','SPONSOR']

display(df_diff_s8[cols].style.apply(colorize, subset=['MOC','MOC_S8','MTC','MTC_S8','SMS','SMS_S8','DATA','MB_S8'], axis=1))

Новые TADIG JT:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S8,MTC_S8,SMS_S8,MB_S8,MOC,MTC,SMS,DATA,LTE_S8,is_discounted,NETWORK_ID,SPONSOR
17,Azerbaijan,Bakcell Limited Liable Company,AZEBC,1.2468,0.0000,0.2340,0.8880,NaN,NaN,NaN,NaN,Yes,NaN,58.0,S1
157,Italy,Telecom Italia SpA,ITASI,0.0636,0.0000,0.1332,9.8580,NaN,NaN,NaN,NaN,Yes,NaN,483.0,S1
213,Panama,Cable & W./Mas Movil,PANCW,1.4244,0.0000,0.4332,1.0320,NaN,NaN,NaN,NaN,No,NaN,708.0,S1
304,Cape Verde,CV Movel,CPVCV,1.5216,0.0000,0.2952,1.0164,NaN,NaN,NaN,NaN,No,NaN,170.0,S2
305,Cape Verde,T+ Telecom,CPVK9,2.0292,0.0000,0.3960,2.0292,NaN,NaN,NaN,NaN,No,NaN,171.0,NaN
306,Cape Verde,T+ Telecom,CPVTM,2.0292,0.0000,0.3960,2.0292,NaN,NaN,NaN,NaN,No,NaN,171.0,NaN
900,Malta,Vodafone - MLTMA,MLTMA,1.0164,1.0164,0.4968,3.0420,NaN,NaN,NaN,NaN,No,NaN,1149.0,NaN
1019,Afghanistan,Roshan/TDCA,AFGK9,0.8928,0.0000,0.3156,0.8880,NaN,NaN,NaN,NaN,No,NaN,7.0,NaN
1020,Afghanistan,Roshan/TDCA,AFGTD,0.8928,0.0000,0.3156,0.8880,NaN,NaN,NaN,NaN,No,NaN,7.0,NaN
1023,Bahamas,Bahamas Telco. Comp.,BHSBH,3.2940,1.0704,0.3576,3.5472,NaN,NaN,NaN,NaN,No,NaN,59.0,NaN



Пропавшие TADIG JT:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID,SPONSOR



Отличия в тарифах JT:


,COUNTRY_NAME,TADIG_CODE,MOC,MOC_S8,MTC,MTC_S8,SMS,SMS_S8,DATA,MB_S8,LTE_S8,is_discounted,NETWORK_ID,SPONSOR


### Подготовить файлы с различающимися и новыми тарифами для загрузки тарифов Partner в OCS

In [15]:
# Подготовить файлы для закрытия тарифов Patner в OCS
# Выводятся пропавшие тарифы и тарифы, которые нужно будет изменить

print('\nPartner OCS to close:')
df_ocs_partner_close = df_ocs_all[
    (df_ocs_all.NETWORK_ID.isin(df_closed_s1.NETWORK_ID.values)    # Тарифы для закрытия
     | (df_ocs_all.NETWORK_ID.isin(df_diff_s1.NETWORK_ID.values))) # Тарифы с изменением
    & (df_ocs_all.SPONSOR_ID == 1)].drop_duplicates() # Только Partner

display(df_ocs_partner_close.head(3))

df_ocs_partner_close.to_csv(join(load_folder, file_close_s1_to_db), index=False)


Partner OCS to close:


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted,HIDDEN,DailyCap,DailyCapLimit,DailyCapTimeZone
2,5570.0,267.0,108.0,1.0,0.660066,0.000000,0.299730,0.0,9.997989,1.0,2017-11-01,None,False,None,False,NaN,NaN
14,5574.0,267.0,383.0,1.0,0.739443,0.949351,0.279886,0.0,10.238187,1.0,2017-11-01,None,False,None,False,NaN,NaN
31,6381.0,267.0,29.0,1.0,0.543096,0.599487,0.249622,0.0,15.000000,1.0,2018-01-01,None,False,None,False,NaN,NaN


In [16]:
# Подготовить файлы с различающимися тарифами для загрузки тарифов Partner в OCS

df_diff_s1['ROAMING_PLAN_RULE_ID'] = ''
df_diff_s1['ROAMING_PLAN_ID']='267'
df_diff_s1['SPONSOR_ID'] = 1
df_diff_s1['MT_SMS_RATE'] = 0
df_diff_s1['CURRENCY_ID'] = 1 #1:USD, 2:EUR
df_diff_s1['START_DATE'] = '2019-10-15 00:00:00'
df_diff_s1['END_DATE'] = 'NULL'

df_diff_s1_100 = df_diff_s1.copy()
df_diff_s1_100[['MOC_S1','MTC_S1','SMS_S1','MB_S1']] = df_diff_s1_100[['MOC_S1','MTC_S1','SMS_S1','MB_S1']] * 2

df_diff_s1[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S1',
'MTC_S1',
'SMS_S1',
'MT_SMS_RATE',
'MB_S1',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder,file_s1_to_db),index=False)

df_diff_s1_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S1',
'MTC_S1',
'SMS_S1',
'MT_SMS_RATE',
'MB_S1',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder,file_s1_to_db_100),index=False)

In [17]:
# Подготовить файлы с новыми тарифами для загрузки тарифов Partner в OCS

df_new_s1['ROAMING_PLAN_RULE_ID'] = ''
df_new_s1['ROAMING_PLAN_ID']='267'
df_new_s1['SPONSOR_ID'] = 1
df_new_s1['MT_SMS_RATE'] = 0
df_new_s1['CURRENCY_ID'] = 1 #1:USD, 2:EUR
df_new_s1['START_DATE'] = '2020-01-01 00:00:00'
df_new_s1['END_DATE'] = 'NULL'

df_new_s1_100 = df_new_s1.copy()
df_new_s1_100[['MOC_S1','MTC_S1','SMS_S1','MB_S1']] = df_new_s1_100[['MOC_S1','MTC_S1','SMS_S1','MB_S1']] * 2

df_new_s1[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S1',
'MTC_S1',
'SMS_S1',
'MT_SMS_RATE',
'MB_S1',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_new_s1_to_db),index=False)

df_new_s1_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S1',
'MTC_S1',
'SMS_S1',
'MT_SMS_RATE',
'MB_S1',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_new_s1_to_db_100), index=False)

### Подготовить файлы с новыми и различающимися тарифами для загрузки тарифов P4 в OCS

In [9]:
# Подготовить файлы для закрытия тарифов P4 в OCS
# Выводятся пропавшие тарифы и тарифы, которые нужно будет изменить

print('\nP4 OCS to close:')
df_ocs_p4_close = df_ocs_all[
    (df_ocs_all.NETWORK_ID.isin(df_closed_s2.NETWORK_ID.values)    # Тарифы для закрытия
     | (df_ocs_all.NETWORK_ID.isin(df_diff_s2.NETWORK_ID.values))) # Тарифы с изменением
    & (df_ocs_all.SPONSOR_ID == 2)].drop_duplicates() # Только P4

display(df_ocs_p4_close.head(3))

df_ocs_p4_close.to_csv(join(load_folder, file_close_s2_to_db), index=False)


P4 OCS to close:


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted,HIDDEN,DailyCap,DailyCapLimit,DailyCapTimeZone
0,4391.0,268.0,171.0,2.0,2.652,0.0,0.2820,0.0,1.5600,1.0,2017-12-01,None,False,None,False,NaN,NaN
196,21816.0,324.0,171.0,2.0,5.304,0.0,0.5640,0.0,3.1200,1.0,2019-01-01,None,False,True,False,NaN,NaN
210,22074.0,324.0,45.0,2.0,0.132,0.0,0.0192,0.0,0.0192,1.0,2019-04-10,None,True,True,False,NaN,NaN


In [21]:
# Подготовить файлы с различающимися тарифами для загрузки тарифов P4 в OCS

df_diff_s2['ROAMING_PLAN_RULE_ID'] = ''
df_diff_s2['ROAMING_PLAN_ID']='268'
df_diff_s2['SPONSOR_ID'] = 2
df_diff_s2['MT_SMS_RATE'] = 0
df_diff_s2['CURRENCY_ID'] = 1 #1:USD, 2:EUR
df_diff_s2['START_DATE'] = '2020-01-01 00:00:00'
df_diff_s2['END_DATE'] = 'NULL'

df_diff_s2_100 = df_diff_s2.copy()
df_diff_s2_100[['MOC_S2','MTC_S2','SMS_S2','MB_S2']] = df_diff_s2_100[['MOC_S2','MTC_S2','SMS_S2','MB_S2']] * 2

df_diff_s2[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S2',
'MTC_S2',
'SMS_S2',
'MT_SMS_RATE',
'MB_S2',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_s2_to_db),index=False)

df_diff_s2_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S2',
'MTC_S2',
'SMS_S2',
'MT_SMS_RATE',
'MB_S2',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_s2_to_db_100), index=False)

In [23]:
# Подготовить файлы с новыми тарифами для загрузки тарифов P4 в OCS

df_new_s2['ROAMING_PLAN_RULE_ID'] = ''
df_new_s2['ROAMING_PLAN_ID']='268'
df_new_s2['SPONSOR_ID'] = 2
df_new_s2['MT_SMS_RATE'] = 0
df_new_s2['CURRENCY_ID'] = 1 #1:USD, 2:EUR
df_new_s2['START_DATE'] = '2020-01-01 00:00:00'
df_new_s2['END_DATE'] = 'NULL'

df_new_s2_100 = df_new_s2.copy()
df_new_s2_100[['MOC_S2','MTC_S2','SMS_S2','MB_S2']] = df_new_s2_100[['MOC_S2','MTC_S2','SMS_S2','MB_S2']] * 2

df_new_s2[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S2',
'MTC_S2',
'SMS_S2',
'MT_SMS_RATE',
'MB_S2',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_new_s2_to_db),index=False)

df_new_s2_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S2',
'MTC_S2',
'SMS_S2',
'MT_SMS_RATE',
'MB_S2',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_new_s2_to_db_100), index=False)

### Подготовить файлы с новыми и различающимися тарифами для загрузки тарифов MB в OCS

In [59]:
# Подготовить файлы для закрытия тарифов MB в OCS
# Выводятся пропавшие тарифы и тарифы, которые нужно будет изменить

print('\nMB OCS to close:')
df_ocs_mb_close = df_ocs_all[
    (df_ocs_all.NETWORK_ID.isin(df_closed_s4.NETWORK_ID.values)    # Тарифы для закрытия
     | (df_ocs_all.NETWORK_ID.isin(df_diff_s4.NETWORK_ID.values))) # Тарифы с изменением
    & (df_ocs_all.SPONSOR_ID == 4)].drop_duplicates() # Только MB

display(df_ocs_mb_close.head(3))

df_ocs_mb_close.to_csv(join(load_folder, file_close_s4_to_db), index=False)


MB OCS to close:


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted,HIDDEN,DailyCap,DailyCapLimit,DailyCapTimeZone
366,24251.0,329.0,499.0,4.0,0.26,0.26,0.26,0.0,0.006,1.0,2019-07-02,None,True,False,False,NaN,NaN
368,24253.0,329.0,821.0,4.0,0.26,0.26,0.26,0.0,0.008,1.0,2019-07-02,None,True,False,False,NaN,NaN
370,24256.0,329.0,470.0,4.0,0.26,0.26,0.26,0.0,0.009,1.0,2019-07-02,None,True,False,False,NaN,NaN


In [60]:
# Подготовить файлы с различающимися тарифами для загрузки тарифов MB в OCS

df_diff_s4['ROAMING_PLAN_RULE_ID'] = ''
df_diff_s4['ROAMING_PLAN_ID']='329'
df_diff_s4['SPONSOR_ID'] = 4
df_diff_s4['MT_SMS_RATE'] = 0
df_diff_s4['CURRENCY_ID'] = 1 #1:USD, 2:EUR
df_diff_s4['START_DATE'] = '2019-07-02 00:00:00'
df_diff_s4['END_DATE'] = 'NULL'

df_diff_s4_100 = df_diff_s4.copy()
df_diff_s4_100[['MOC_S4','MTC_S4','SMS_S4','MB_S4']] = df_diff_s4_100[['MOC_S4','MTC_S4','SMS_S4','MB_S4']] * 2

df_diff_s4[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S4',
'MTC_S4',
'SMS_S4',
'MT_SMS_RATE',
'MB_S4',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_s4_to_db),index=False)

df_diff_s4_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S4',
'MTC_S4',
'SMS_S4',
'MT_SMS_RATE',
'MB_S4',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_s4_to_db_100), index=False)

In [61]:
# Подготовить файлы с новыми тарифами для загрузки тарифов MB в OCS

df_new_s4['ROAMING_PLAN_RULE_ID'] = ''
df_new_s4['ROAMING_PLAN_ID']='329'
df_new_s4['SPONSOR_ID'] = 4
df_new_s4['MT_SMS_RATE'] = 0
df_new_s4['CURRENCY_ID'] = 1 #1:USD, 2:EUR
df_new_s4['START_DATE'] = '2019-07-02 00:00:00'
df_new_s4['END_DATE'] = 'NULL'

df_new_s4_100 = df_new_s4.copy()
df_new_s4_100[['MOC_S4','MTC_S4','SMS_S4','MB_S4']] = df_new_s4_100[['MOC_S4','MTC_S4','SMS_S4','MB_S4']] * 2

df_new_s4[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S4',
'MTC_S4',
'SMS_S4',
'MT_SMS_RATE',
'MB_S4',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_new_s4_to_db),index=False)

df_new_s4_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S4',
'MTC_S4',
'SMS_S4',
'MT_SMS_RATE',
'MB_S4',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_new_s4_to_db_100), index=False)

### Подготовить файлы с новыми и различающимися тарифами для загрузки тарифов SMART в OCS

In [12]:
# Подготовить файлы со всеми тарифами из файла для загрузки тарифов Smart в OCS Clone

df_all_s5_clone = df_smart_net_id.copy()

df_all_s5_clone['ROAMING_PLAN_RULE_ID'] = ''
df_all_s5_clone['ROAMING_PLAN_ID']=''
df_all_s5_clone['SPONSOR_ID'] = 5
df_all_s5_clone['MT_SMS_RATE'] = 0
df_all_s5_clone['CURRENCY_ID'] = 1 #1:USD, 2:EUR
df_all_s5_clone['START_DATE'] = '2019-10-15 00:00:00'
df_all_s5_clone['END_DATE'] = 'NULL'
df_all_s5_clone['is_discounted'] = 0

df_all_s5_clone_100 = df_all_s5_clone.copy()
df_all_s5_clone_100[['MOC','MTC','SMS','MB']] = df_all_s5_clone_100[['MOC','MTC','SMS','MB']] * smart_price_to_cost_rate
df_all_s5 = df_all_s5_clone_100.copy()
df_all_s5_100 = df_all_s5.copy()
df_all_s5_100[['MOC','MTC','SMS','MB']] = df_all_s5_100[['MOC','MTC','SMS','MB']] * 2

df_all_s5_clone[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC',
'MTC',
'SMS',
'MT_SMS_RATE',
'MB',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_new_s5_to_db_clone),index=False)

df_all_s5_clone_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC',
'MTC',
'SMS',
'MT_SMS_RATE',
'MB',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_new_s5_to_db_clone_100),index=False)

df_all_s5[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC',
'MTC',
'SMS',
'MT_SMS_RATE',
'MB',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_new_s5_to_db),index=False)

df_all_s5_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC',
'MTC',
'SMS',
'MT_SMS_RATE',
'MB',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder, file_new_s5_to_db_100),index=False)

### Новые тарифы SMART для загрузки в RoamDb

In [13]:
df_all_s5_clone_100[['TADIG_CODE','MOC','MTC','SMS','MB'
                     ,'Voice_Increment_SEC','Data_Increment_KB','is_discounted','LTE_S5']].head()

,TADIG_CODE,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,is_discounted,LTE_S5
0,ALBVF,0.1610,0.0000,0.008625,0.008625,60.0,1.0,0,No
1,NaN,0.5750,0.0575,0.057500,0.057500,60.0,1.0,0,No
2,NaN,0.5750,0.0575,0.057500,0.057500,60.0,1.0,0,No
3,ARGCM,0.4025,0.0345,0.011500,0.009200,60.0,1.0,0,Yes
4,ARGTP,0.2875,0.0000,0.011500,0.011500,60.0,1.0,0,Yes


In [14]:
columns_origin = ['TADIG_CODE',
  'MOC',
  'MTC',
  'SMS',
  'MB',
  'Voice_Increment_SEC',
  'Data_Increment_KB',
  'is_discounted',
  'LTE_S5']

columns_roamdb = ['TADIG CODE',
 'Mobile Origination',
 'Mobile Termination',
 'SMS',
 'DATA',
 'Voice Increment',
 'Data Increment',
 'IsDiscounted (Yes/No)',
 'Support4G(Yes/No)',]

df_s5_roamdb = df_all_s5_clone_100[columns_origin].rename(dict(zip(columns_origin,columns_roamdb)), axis=1)
df_s5_roamdb[columns_roamdb].to_csv(join(load_folder,file_s5_roamdb),index=False)
df_s5_roamdb.head()

,TADIG CODE,Mobile Origination,Mobile Termination,SMS,DATA,Voice Increment,Data Increment,IsDiscounted (Yes/No),Support4G(Yes/No)
0,ALBVF,0.1610,0.0000,0.008625,0.008625,60.0,1.0,0,No
1,NaN,0.5750,0.0575,0.057500,0.057500,60.0,1.0,0,No
2,NaN,0.5750,0.0575,0.057500,0.057500,60.0,1.0,0,No
3,ARGCM,0.4025,0.0345,0.011500,0.009200,60.0,1.0,0,Yes
4,ARGTP,0.2875,0.0000,0.011500,0.011500,60.0,1.0,0,Yes


### Новые тарифы P4 для загрузки в RoamDb

In [38]:
# Новые тарифы P4 для загрузки в RoamDb

# !!! Тарифы пересчитыны а USD, тогда как в RoamDB тарифы в EUR.
# В качестве быстрого решения можно поменять Currency

file_partner_roamdb = 'p4_roamdb_result.csv'

df_temp = df_s1_s2_s4_sponsor.copy()

print("Новые TADIG P4:")
cols_p4 = ['TADIG_CODE','MOC_S2','MTC_S2','SMS_S2','MB_S2'
           ,'P4_LTE','Voice_Increment_SEC_S2','Data_Increment_KB_S2']

df_rdb = df_temp.loc[((df_temp.MB_S2.notnull())
                 & (df_temp.TADIG_CODE.notnull())), cols_p4]

df_rdb['IsDiscounted (Yes/No)'] = 'Yes'
df_rdb.rename({'TADIG_CODE':'TADIG CODE','MOC_S2':'Mobile Origination','MTC_S2':'Mobile Termination','SMS_S2':'SMS','MB_S2':'DATA',
          'Voice_Increment_SEC_S2':'Voice Increment','Data_Increment_KB_S2':'Data Increment','P4_LTE':'Support4G(Yes/No)'},
         axis=1,inplace=True)

df_rdb['Support4G(Yes/No)'].replace({'yes':'Yes','no':'No'}, inplace=True)

columns_ordered = ['TADIG CODE',
 'Mobile Origination',
 'Mobile Termination',
 'SMS',
 'DATA',
 'Voice Increment',
 'Data Increment',
 'IsDiscounted (Yes/No)',
 'Support4G(Yes/No)',]

df_rdb[columns_ordered].to_csv(join(load_folder,file_partner_roamdb), index=False)

df_rdb[columns_ordered].head()

Новые TADIG P4:


,TADIG CODE,Mobile Origination,Mobile Termination,SMS,DATA,Voice Increment,Data Increment,IsDiscounted (Yes/No),Support4G(Yes/No)
9,AUSOP,0.06325,0.0,0.00920,0.009200,60,1,Yes,YES
10,AUTCA,0.02300,0.0,0.00345,0.001437,1,1,Yes,NO
11,AUTHU,0.02300,0.0,0.00345,0.001437,1,1,Yes,NO
12,AUTCA,0.02300,0.0,0.00345,0.001437,1,1,Yes,YES
13,AUTHU,0.02300,0.0,0.00345,0.001437,1,1,Yes,YES


### Прочее
Prepaid Packages для TOT сделаны на основе тарифов.
См: c:\W_DATA_ROAM\ПРОЕКТЫ_NEW_CONNECTION\NEW_TOGO_CONNECTION_TOT\800_PrepaidPackages\

In [2]:
df_p4 = pd.read_csv(join(load_folder, 'P4_20190801.csv'))
df_p4.head()

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,P4_CAMEL,P4_LTE
0,Albania,TELEKOM ALBANIA SH.A,ALBAM,0.449,NaN,0.052,0.04000,1,1 kB,yes,yes
1,Albania,ALBtelecom sh.a.,ALBEM,0.390,NaN,0.040,0.03900,60,1 kB,yes,yes
2,Armenia,Armenia Telephone Company (ArmenTel),ARM01,0.585,NaN,0.015,0.00400,60,1 kB,yes,no
3,Australia,SingTel Optus Pty Limited,AUSOP,0.055,NaN,0.008,0.00800,60,1 kB,yes,yes
4,Austria,Hutchison Drei Austria GmbH,AUTHU,0.020,NaN,0.003,0.00125,1,1 kB,yes,no


In [4]:
p4_block_list = ['ARM01', 'BGDBL', 'GEOMT', 'KAZKT', 'PAKMK', 'PAKWA', 'RUSBD',  'UKRKS', 'UKRRS',
                 'BGDGP', 'BGRCM', 'DNKDM', 'HUNH1', 'MMRTN', 'MNEPM', 'MYSMT', 'NORTM', 'PAKTP',
                 'SWECN', 'SWEEP', 'THADT', 'YUGMT', 'YUGPM', 'NORMC', 'CHECX', 'BGRCM', 'CZEET',
                 'HUNH1', 'MNEPM', 'YUGMT', 'SVKO2']

In [8]:
df_p4[~df_p4.TADIG.isin(p4_block_list)].to_csv(join(load_folder, 'P4_20200101.csv'), index=False)